In [259]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, gc
import math
warnings.filterwarnings('ignore')

In [260]:
gc.collect()

0

In [261]:
movies = pd.read_csv('movie_metadata.csv')
movies = movies.drop(columns=['num_critic_for_reviews','director_facebook_likes','actor_3_facebook_likes',
                              'actor_1_facebook_likes','num_voted_users','cast_total_facebook_likes',
                              'actor_2_facebook_likes','aspect_ratio','movie_facebook_likes',
                              'facenumber_in_poster'])


movies['cast_information'] = movies[['actor_1_name', 'actor_2_name', 'actor_3_name']].fillna('').agg(','.join, axis=1)
movies = movies.drop(columns=['actor_1_name', 'actor_2_name', 'actor_3_name'])


In [262]:
for i in range(movies.shape[0]):
    duration = movies['duration'].iloc[i]

    if pd.isna(duration):
        movies['duration'].iloc[i] = 'NaN'
        continue
    else:
        # Ensure the duration is numeric
        try:
            time = int(duration)
            hours = time // 60
            minutes = time % 60
            movies['duration'].iloc[i] = f"{hours}h {minutes}m"
        except:
            movies['duration'].iloc[i] = 'NaN'


In [263]:

movies['genres'] = movies['genres'].str.split('|')
movies = movies.explode('genres').reset_index(drop=True)


In [264]:
movies['movie_title'] = movies['movie_title'].astype(str)
movies['movie_title'] = movies['movie_title'].str.strip()
movies['movie_title']  = movies['movie_title'].str.replace('\xa0', '')
movies['movie_title']  = movies['movie_title'].str.replace('\n', '')
movies['movie_title']  = movies['movie_title'].str.replace('\t', '')
movies['movie_title']  = movies['movie_title'].str.replace('(', '')
movies['movie_title']  = movies['movie_title'].str.replace(')', '')
movies['movie_title']  = movies['movie_title'].str.replace(':', '')
movies['movie_title']  = movies['movie_title'].str.replace('.', '')
movies['movie_title']  = movies['movie_title'].str.replace(',', '')
movies['movie_title']  = movies['movie_title'].str.replace('?', '')
movies['movie_title']  = movies['movie_title'].str.replace('!', '')
movies['movie_title']  = movies['movie_title'].str.replace('"', '')
movies['movie_title']  = movies['movie_title'].str.replace("'", '')


In [265]:
movie_no_release_year = set(movies['movie_title'].iloc[np.where(movies['title_year'].isnull())].values)

In [266]:
temp_list = [
    "10,000 B.C. || 2008",
"12 Monkeys || 1995",
"3rd Rock from the Sun || 1996",
"A Touch of Frost || 1992",
"Anger Management || 2012",
"Animal Kingdom || 2010",
"Anne of Green Gables || 1985",
"Arthur || 1996",
"Bewitched || 1964",
"Bones || 2005",
"BrainDead || 2016",
"Buffy the Vampire Slayer || 1997",
"Carlos || 2010",
"Constantine || 2014",
"Creature || 1998",
"Daredevil || 2015",
"Dark Angel || 2000",
"Deadline Gallipoli || 2015",
"Defiance || 2013",
"Dekalog || 1989",
"Del 1 - Män som hatar kvinnor || 2009",
"Emma || 2009",
"Empire || 2015",
"Entourage || 2004",
"Eureka || 2006",
"Fargo || 2014",
"Fired Up || 2009",
"Friday Night Lights || 2006",
"Get Real || 1999",
"Ghost Hunters || 2004",
"Gomorrah || 2014",
"Gone Baby Gone || 2007",
"Hannibal || 2013",
"Happy Valley || 2014",
"Heroes || 2006",
"Hit the Floor || 2013",
"Home Movies || 1999",
"In the Heat of the Night || 1988",
"It's Always Sunny in Philadelphia || 2005",
"Jesse || 1998",
"Last Man Standing || 2011",
"Life || 2007",
"Lilyhammer || 2012",
"Limitless || 2015",
"Lovesick || 2014",
"Luther || 2010",
"M*A*S*H || 1972",
"McHale's Navy || 1962",
"Meet the Browns || 2009",
"Miami Vice || 1984",
"Nikita || 2010",
"Outlander || 2014",
"Perception || 2012",
"Preacher || 2016",
"Psych || 2006",
"Revolution || 2012",
"Robot Chicken || 2005",
"Rogue || 2013",
"Rules of Engagement || 2007",
"Rush Hour || 1998",
"Sabrina, the Teenage Witch || 1996",
"Saving Grace || 2007",
"Scream: The TV Series || 2015",
"Secrets and Lies || 2015",
"Sex and the City || 1998",
"Shaun the Sheep || 2007",
"Sleepy Hollow || 2013",
"Sonny with a Chance || 2009",
"Space: Above and Beyond || 1995",
"Spartacus: War of the Damned || 2013",
"Star Wars: Episode VII - The Force Awakens || 2015",
"Star Wars: The Clone Wars || 2008",
"Stargate SG-1 || 1997",
"Strangers with Candy || 1999",
"The A-Team || 1983",
"The Bachelor || 2002",
"The Bold and the Beautiful || 1987",
"The Border || 2008",
"The Company || 2007",
"The Dead Zone || 2002",
"The Doombolt Chase || 1978",
"The Family || 2016",
"The Following || 2013",
"The Girlfriend Experience || 2016",
"The Grand || 1997",
"The Honeymooners || 1955",
"The Inbetweeners || 2008",
"The Messengers || 2015",
"The Missing || 2014",
"The O.C. || 2003",
"The Player || 2015",
"The Powerpuff Girls || 1998",
"The Returned || 2012",
"The Secret || 2006",
"The Streets of San Francisco || 1972",
"The Towering Inferno || 1974",
"Trapped || 2015",
"Twisted || 2013",
"Unforgettable || 2011",
"Unforgotten || 2015",
"Veronica Mars || 2004",
"War & Peace || 2016",
"Wings || 1990",
"Wolf Creek || 2016",
"Wuthering Heights || 2009",
"Yu-Gi-Oh! Duel Monsters || 2000"

]
movie_title = []
release_year = []
for i in range(len(temp_list)):
  stripped_info = temp_list[i].split(" || ")
  movie_title.append(stripped_info[0])
  release_year.append(stripped_info[1])

In [267]:
# Apply the same cleaning to the movie titles from temp_list
cleaned_movie_titles = []
for title in movie_title:
    cleaned_title = title.replace('\xa0', '').replace('\n', '').replace('\t', '')
    cleaned_title = cleaned_title.replace('(', '').replace(')', '').replace(':', '').replace('.', '')
    cleaned_title = cleaned_title.replace(',', '').replace('?', '').replace('!', '').replace('"', '').replace("'", '')
    # Also remove hyphens from the temp_list titles to match the cleaning on the dataframe
    cleaned_title = cleaned_title.replace('-', '')
    cleaned_movie_titles.append(cleaned_title.strip())


for i in range(movies.shape[0]):
  title = movies['movie_title'].iloc[i]
  title = title.strip()
  if title in movie_no_release_year:
    print("title in the set = ",title)
    # Find the index in the cleaned list
    try:
        # Use the cleaned_movie_titles list for finding the index
        idx = cleaned_movie_titles.index(title)
        movies['title_year'].iloc[i] = release_year[idx]
    except ValueError:
        # Handle cases where the title is still not found after cleaning
        print(f"Warning: Could not find a match for '{title}' in the release year list after cleaning.")
  else:
    continue

title in the set =  Star Wars Episode VII - The Force Awakens
title in the set =  Miami Vice
title in the set =  Miami Vice
title in the set =  Miami Vice
title in the set =  Miami Vice
title in the set =  Miami Vice
title in the set =  The A-Team
title in the set =  The A-Team
title in the set =  The A-Team
title in the set =  10000 BC
title in the set =  Hannibal
title in the set =  Hannibal
title in the set =  Hannibal
title in the set =  Hannibal
title in the set =  Hannibal
title in the set =  Daredevil
title in the set =  Daredevil
title in the set =  Daredevil
title in the set =  Daredevil
title in the set =  Daredevil
title in the set =  Daredevil
title in the set =  Bewitched
title in the set =  Bewitched
title in the set =  Bewitched
title in the set =  Constantine
title in the set =  Constantine
title in the set =  Constantine
title in the set =  Constantine
title in the set =  Life
title in the set =  Life
title in the set =  Life
title in the set =  Sleepy Hollow
title in 

In [268]:
for i in range(movies.shape[0]):
  year = movies['title_year'].iloc[i]
  if pd.isna(year):
    movies['title_year'].iloc[i] = 'NaN'
  else:
    movies['title_year'].iloc[i] = int(year)
movies

,color,director_name,duration,gross,genres,movie_title,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,imdb_score,cast_information
0,Color,James Cameron,2h 58m,760505847.0,Action,Avatar,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009,7.9,"CCH Pounder,Joel David Moore,Wes Studi"
1,Color,James Cameron,2h 58m,760505847.0,Adventure,Avatar,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009,7.9,"CCH Pounder,Joel David Moore,Wes Studi"
2,Color,James Cameron,2h 58m,760505847.0,Fantasy,Avatar,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009,7.9,"CCH Pounder,Joel David Moore,Wes Studi"
3,Color,James Cameron,2h 58m,760505847.0,Sci-Fi,Avatar,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009,7.9,"CCH Pounder,Joel David Moore,Wes Studi"
4,Color,Gore Verbinski,2h 49m,309404152.0,Action,Pirates of the Caribbean At Worlds End,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007,7.1,"Johnny Depp,Orlando Bloom,Jack Davenport"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14499,Color,Benjamin Roberds,1h 16m,NaN,Thriller,A Plague So Pleasant,NaN,http://www.imdb.com/title/tt2107644/?ref_=fn_t...,3.0,English,USA,NaN,1400.0,2013,6.3,"Eva Boehnke,Maxwell Moody,David Chandler"
14500,Color,Daniel Hsia,1h 40m,10443.0,Comedy,Shanghai Calling,NaN,http://www.imdb.com/title/tt2070597/?ref_=fn_t...,9.0,English,USA,PG-13,NaN,2012,6.3,"Alan Ruck,Daniel Henney,Eliza Coupe"
14501,Color,Daniel Hsia,1h 40m,10443.0,Drama,Shanghai Calling,NaN,http://www.imdb.com/title/tt2070597/?ref_=fn_t...,9.0,English,USA,PG-13,NaN,2012,6.3,"Alan Ruck,Daniel Henney,Eliza Coupe"
14502,Color,Daniel Hsia,1h 40m,10443.0,Romance,Shanghai Calling,NaN,http://www.imdb.com/title/tt2070597/?ref_=fn_t...,9.0,English,USA,PG-13,NaN,2012,6.3,"Alan Ruck,Daniel Henney,Eliza Coupe"


# My past data


In [273]:
another_movies = pd.read_csv('movies_final_data.csv')
another_movies['genres_included'] = another_movies['genres_included'].str.split(',')
another_movies = another_movies.explode('genres_included').reset_index(drop=True)
for i in range(another_movies.shape[0]):
  time = another_movies['runtime'].iloc[i]
  if pd.isna(time):
    another_movies['runtime'].iloc[i] = 'NaN'
  else:
    time = int(time)
    hours = time // 60
    minutes = time % 60
    another_movies['runtime'].iloc[i] = f"{hours}h {minutes}m"

  rel_date = another_movies['release_date'].iloc[i]
  if pd.isna(rel_date):
    another_movies['release_date'].iloc[i] = 'NaN'
  else:
    rel_data = rel_date.split('-')
    another_movies['release_date'].iloc[i] = rel_data[0]
another_movies

,adult,genres,id,imdb_id,original_language,original_title,overview,popularity,release_date,runtime,spoken_languages,tagline,title,vote_average,vote_count,genres_included,available_languages,cast_info_complete
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,114709.0,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,1h 21m,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0,Animation,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
1,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,114709.0,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,1h 21m,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0,Comedy,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
2,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,114709.0,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,1h 21m,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0,Family,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
3,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,113497.0,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995,1h 44m,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,Adventure,"English(en),Français(fr)","Robin Williams(Alan Parrish),Jonathan Hyde(Sam..."
4,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,113497.0,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995,1h 44m,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,Fantasy,"English(en),Français(fr)","Robin Williams(Alan Parrish),Jonathan Hyde(Sam..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93531,False,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,303758.0,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003,1h 30m,"[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0,Action,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93532,False,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,303758.0,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003,1h 30m,"[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0,Drama,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93533,False,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,303758.0,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003,1h 30m,"[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0,Thriller,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93534,False,[],227506,8536.0,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917,1h 27m,[],NaN,Satan Triumphant,0.0,0.0,NaN,NaN,"Markie Adams(Lauren),Roberto Aguire(Diego),Tin..."


In [274]:
h_m = {}
for i in range(another_movies.shape[0]):
  language = another_movies['available_languages'].iloc[i]
  if pd.isna(language):
    continue
  splitted_lang = language.split(',')
  for j in range(len(splitted_lang)):
    lang = splitted_lang[j].split('(')
    if lang[0]  not in h_m.keys():
      h_m[lang[1][:-1]] = lang[0]


In [275]:
for i in range(another_movies.shape[0]):
  lang = another_movies['original_language'].iloc[i]
  av_lang = another_movies['available_languages'].iloc[i]
  if pd.isna(lang):
    continue
  if lang in h_m.keys():
    another_movies['original_language'].iloc[i] = h_m[lang] +'('+lang +')'
another_movies

,adult,genres,id,imdb_id,original_language,original_title,overview,popularity,release_date,runtime,spoken_languages,tagline,title,vote_average,vote_count,genres_included,available_languages,cast_info_complete
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,114709.0,English(en),Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,1h 21m,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0,Animation,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
1,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,114709.0,English(en),Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,1h 21m,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0,Comedy,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
2,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,114709.0,English(en),Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,1h 21m,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0,Family,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
3,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,113497.0,English(en),Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995,1h 44m,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,Adventure,"English(en),Français(fr)","Robin Williams(Alan Parrish),Jonathan Hyde(Sam..."
4,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,113497.0,English(en),Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995,1h 44m,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,Fantasy,"English(en),Français(fr)","Robin Williams(Alan Parrish),Jonathan Hyde(Sam..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93531,False,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,303758.0,English(en),Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003,1h 30m,"[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0,Action,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93532,False,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,303758.0,English(en),Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003,1h 30m,"[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0,Drama,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93533,False,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,303758.0,English(en),Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003,1h 30m,"[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0,Thriller,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93534,False,[],227506,8536.0,English(en),Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917,1h 27m,[],NaN,Satan Triumphant,0.0,0.0,NaN,NaN,"Markie Adams(Lauren),Roberto Aguire(Diego),Tin..."


In [276]:
gc.collect()

84

In [277]:

another_movies = another_movies.drop(columns=['genres','id','imdb_id','popularity','vote_count','spoken_languages'])
another_movies

,adult,original_language,original_title,overview,release_date,runtime,tagline,title,vote_average,genres_included,available_languages,cast_info_complete
0,False,English(en),Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995,1h 21m,NaN,Toy Story,7.7,Animation,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
1,False,English(en),Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995,1h 21m,NaN,Toy Story,7.7,Comedy,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
2,False,English(en),Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995,1h 21m,NaN,Toy Story,7.7,Family,English(en),"Tom Hanks(Woody (voice)),Tim Allen(Buzz Lighty..."
3,False,English(en),Jumanji,When siblings Judy and Peter discover an encha...,1995,1h 44m,Roll the dice and unleash the excitement!,Jumanji,6.9,Adventure,"English(en),Français(fr)","Robin Williams(Alan Parrish),Jonathan Hyde(Sam..."
4,False,English(en),Jumanji,When siblings Judy and Peter discover an encha...,1995,1h 44m,Roll the dice and unleash the excitement!,Jumanji,6.9,Fantasy,"English(en),Français(fr)","Robin Williams(Alan Parrish),Jonathan Hyde(Sam..."
...,...,...,...,...,...,...,...,...,...,...,...,...
93531,False,English(en),Betrayal,"When one of her hits goes wrong, a professiona...",2003,1h 30m,A deadly game of wits.,Betrayal,3.8,Action,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93532,False,English(en),Betrayal,"When one of her hits goes wrong, a professiona...",2003,1h 30m,A deadly game of wits.,Betrayal,3.8,Drama,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93533,False,English(en),Betrayal,"When one of her hits goes wrong, a professiona...",2003,1h 30m,A deadly game of wits.,Betrayal,3.8,Thriller,English(en),"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)..."
93534,False,English(en),Satana likuyushchiy,"In a small town live two brothers, one a minis...",1917,1h 27m,NaN,Satan Triumphant,0.0,NaN,NaN,"Markie Adams(Lauren),Roberto Aguire(Diego),Tin..."


In [278]:
another_movies.columns

Index(['adult', 'original_language', 'original_title', 'overview',
       'release_date', 'runtime', 'tagline', 'title', 'vote_average',
       'genres_included', 'available_languages', 'cast_info_complete'],
      dtype='object')

In [279]:
for i in range(movies.shape[0]):
  director = movies['director_name'].iloc[i]
  if pd.isna(director):
    continue
  else:
    director = director.replace('\xa0', '').replace('\n', '').replace('\t', '')
    director = director.replace('(', '').replace(')', '').replace(':', '').replace('.', '')
    director = director.replace(',', '').replace('?', '').replace('!', '').replace('"', '').replace("'", '')
    director = director.replace('-', '')
    director = director.strip()
    director += "(director)"
    movies['cast_information'].iloc[i] += ',' + director




In [280]:
movies = movies.drop(columns=['director_name','color','gross','plot_keywords','movie_imdb_link','num_user_for_reviews'])
movies

,duration,genres,movie_title,language,country,content_rating,budget,title_year,imdb_score,cast_information
0,2h 58m,Action,Avatar,English,USA,PG-13,237000000.0,2009,7.9,"CCH Pounder,Joel David Moore,Wes Studi,James C..."
1,2h 58m,Adventure,Avatar,English,USA,PG-13,237000000.0,2009,7.9,"CCH Pounder,Joel David Moore,Wes Studi,James C..."
2,2h 58m,Fantasy,Avatar,English,USA,PG-13,237000000.0,2009,7.9,"CCH Pounder,Joel David Moore,Wes Studi,James C..."
3,2h 58m,Sci-Fi,Avatar,English,USA,PG-13,237000000.0,2009,7.9,"CCH Pounder,Joel David Moore,Wes Studi,James C..."
4,2h 49m,Action,Pirates of the Caribbean At Worlds End,English,USA,PG-13,300000000.0,2007,7.1,"Johnny Depp,Orlando Bloom,Jack Davenport,Gore ..."
...,...,...,...,...,...,...,...,...,...,...
14499,1h 16m,Thriller,A Plague So Pleasant,English,USA,NaN,1400.0,2013,6.3,"Eva Boehnke,Maxwell Moody,David Chandler,Benja..."
14500,1h 40m,Comedy,Shanghai Calling,English,USA,PG-13,NaN,2012,6.3,"Alan Ruck,Daniel Henney,Eliza Coupe,Daniel Hsi..."
14501,1h 40m,Drama,Shanghai Calling,English,USA,PG-13,NaN,2012,6.3,"Alan Ruck,Daniel Henney,Eliza Coupe,Daniel Hsi..."
14502,1h 40m,Romance,Shanghai Calling,English,USA,PG-13,NaN,2012,6.3,"Alan Ruck,Daniel Henney,Eliza Coupe,Daniel Hsi..."


we need movie title, genre, rating, duration,cast, original language, certificate,release year,

In [281]:
# for i in range(movies.shape[0]):
#   lang = movies['language'].iloc[i]
#   if pd.isna(lang):
#     continue
#   elif lang in h_m.values():
#     print("lang in values = ",lang)
#     total_lang = lang+'('+h_m[lang]+')'
#     movies['language'].iloc[i] = total_lang
# movies

lang in values =  English


KeyError: 'English'

In [282]:
from typing_extensions import final

final_movie_title = []
final_movie_rating =[]
final_movie_duration = []
final_movie_genre = []
final_movie_release_year = []
final_movie_cast = []
final_movie_language = []
final_movie_certificate = []
final_available_languages = []
final_overview = []

for i in range(movies.shape[0]):
  final_movie_title.append(movies['movie_title'].iloc[i])
  final_movie_rating.append(movies['imdb_score'].iloc[i])
  final_movie_duration.append(movies['duration'].iloc[i])
  final_movie_genre.append(movies['genres'].iloc[i])
  final_movie_release_year.append(movies['title_year'].iloc[i])
  final_movie_cast.append(movies['cast_information'].iloc[i])
  final_movie_language.append(movies['language'].iloc[i])
  final_movie_certificate.append(movies['content_rating'].iloc[i])
  final_available_languages.append(movies['language'].iloc[i])
  final_overview.append('NaN')

for i in range(another_movies.shape[0]):
  final_movie_title.append(another_movies['title'].iloc[i])
  final_movie_rating.append(another_movies['vote_average'].iloc[i])
  final_movie_duration.append(another_movies['runtime'].iloc[i])
  final_movie_genre.append(another_movies['genres_included'].iloc[i])
  final_movie_release_year.append(another_movies['release_date'].iloc[i])
  final_movie_cast.append(another_movies['cast_info_complete'].iloc[i])
  final_movie_language.append(another_movies['original_language'].iloc[i])
  final_movie_certificate.append(another_movies['adult'].iloc[i])
  final_available_languages.append(another_movies['available_languages'].iloc[i])
  final_overview.append(another_movies['overview'].iloc[i])

final_data = {
    'Movie Title': final_movie_title,
    'IMDB Ratings': final_movie_rating,
    'Duration': final_movie_duration,
    'Genre': final_movie_genre,
    'Release Year': final_movie_release_year,
    'Cast and Crew': final_movie_cast,
    'Original Language': final_movie_language,
    'Certificate (or) Adult Rated Information': final_movie_certificate,
    'Also available in': final_available_languages,
    'Overview': final_overview
}

final_df = pd.DataFrame(final_data)
final_df

,Movie Title,IMDB Ratings,Duration,Genre,Release Year,Cast and Crew,Original Language,Certificate (or) Adult Rated Information,Also available in,Overview
0,Avatar,7.9,2h 58m,Action,2009,"CCH Pounder,Joel David Moore,Wes Studi,James C...",English,PG-13,English,NaN
1,Avatar,7.9,2h 58m,Adventure,2009,"CCH Pounder,Joel David Moore,Wes Studi,James C...",English,PG-13,English,NaN
2,Avatar,7.9,2h 58m,Fantasy,2009,"CCH Pounder,Joel David Moore,Wes Studi,James C...",English,PG-13,English,NaN
3,Avatar,7.9,2h 58m,Sci-Fi,2009,"CCH Pounder,Joel David Moore,Wes Studi,James C...",English,PG-13,English,NaN
4,Pirates of the Caribbean At Worlds End,7.1,2h 49m,Action,2007,"Johnny Depp,Orlando Bloom,Jack Davenport,Gore ...",English,PG-13,English,NaN
...,...,...,...,...,...,...,...,...,...,...
108035,Betrayal,3.8,1h 30m,Action,2003,"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)...",English(en),False,English(en),"When one of her hits goes wrong, a professiona..."
108036,Betrayal,3.8,1h 30m,Drama,2003,"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)...",English(en),False,English(en),"When one of her hits goes wrong, a professiona..."
108037,Betrayal,3.8,1h 30m,Thriller,2003,"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)...",English(en),False,English(en),"When one of her hits goes wrong, a professiona..."
108038,Satan Triumphant,0.0,1h 27m,NaN,1917,"Markie Adams(Lauren),Roberto Aguire(Diego),Tin...",English(en),False,NaN,"In a small town live two brothers, one a minis..."


In [288]:
for i in range(final_df.shape[0]):
  lang = final_df['Original Language'].iloc[i]
  av_lang = final_df['Also available in'].iloc[i]
  if pd.isna(lang):
    continue
  else:
    if lang == 'English':
      final_df['Original Language'].iloc[i] = 'English(en)'
    if not pd.isna(av_lang):
      final_df['Also available in'].iloc[i] = 'English(en)'
final_df

,Movie Title,IMDB Ratings,Duration,Genre,Release Year,Cast and Crew,Original Language,Certificate (or) Adult Rated Information,Also available in,Overview
0,Avatar,7.9,2h 58m,Action,2009,"CCH Pounder,Joel David Moore,Wes Studi,James C...",English(en),PG-13,English(en),NaN
1,Avatar,7.9,2h 58m,Adventure,2009,"CCH Pounder,Joel David Moore,Wes Studi,James C...",English(en),PG-13,English(en),NaN
2,Avatar,7.9,2h 58m,Fantasy,2009,"CCH Pounder,Joel David Moore,Wes Studi,James C...",English(en),PG-13,English(en),NaN
3,Avatar,7.9,2h 58m,Sci-Fi,2009,"CCH Pounder,Joel David Moore,Wes Studi,James C...",English(en),PG-13,English(en),NaN
4,Pirates of the Caribbean At Worlds End,7.1,2h 49m,Action,2007,"Johnny Depp,Orlando Bloom,Jack Davenport,Gore ...",English(en),PG-13,English(en),NaN
...,...,...,...,...,...,...,...,...,...,...
108035,Betrayal,3.8,1h 30m,Action,2003,"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)...",English(en),False,English(en),"When one of her hits goes wrong, a professiona..."
108036,Betrayal,3.8,1h 30m,Drama,2003,"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)...",English(en),False,English(en),"When one of her hits goes wrong, a professiona..."
108037,Betrayal,3.8,1h 30m,Thriller,2003,"Sridevi Kapoor(Devki Sabarwal),Sajal Ali(Arya)...",English(en),False,English(en),"When one of her hits goes wrong, a professiona..."
108038,Satan Triumphant,0.0,1h 27m,NaN,1917,"Markie Adams(Lauren),Roberto Aguire(Diego),Tin...",English(en),False,NaN,"In a small town live two brothers, one a minis..."


In [289]:
final_df.to_csv('Final Movie Dataset.csv', index=False)